In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## UNIR DOS TABLAS

In [2]:
# Cargar el dataset (ajusta el nombre del archivo)
df = pd.read_csv("../data/all_shot_data_final.csv")
df2 = pd.read_csv("../data/NBA Stats 202324 All Stats  NBA Player Props Tool.csv")
# Mostrar nombres de columnas
print(df.columns)

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM', 'player_name', 'Season Type', 'HOME_TEAM_NAME',
       'AWAY_TEAM_NAME', 'LOCATION'],
      dtype='object')


In [3]:
df.head(2)

GRID_TYPE   GAME_ID  GAME_EVENT_ID  PLAYER_ID       PLAYER_NAME  \
0  Shot Chart Detail  22300031            130    1630173  Precious Achiuwa   
1  Shot Chart Detail  22300031            148    1630173  Precious Achiuwa   

      TEAM_ID        TEAM_NAME  PERIOD  MINUTES_REMAINING  SECONDS_REMAINING  \
0  1610612761  Toronto Raptors       1                  1                 49   
1  1610612761  Toronto Raptors       1                  0                 11   

   ... SHOT_ATTEMPTED_FLAG SHOT_MADE_FLAG GAME_DATE  HTM  VTM  \
0  ...                   1              1  20231117  TOR  BOS   
1  ...                   1              0  20231117  TOR  BOS   

        player_name     Season Type  HOME_TEAM_NAME  AWAY_TEAM_NAME  LOCATION  
0  Precious Achiuwa  Regular Season             BOS             TOR      AWAY  
1  Precious Achiuwa  Regular Season             BOS             TOR      AWAY  

[2 rows x 29 columns]

In [4]:
# UNIR TABLAS
df = df.merge(df2[['NAME', 'POS']],
              left_on="PLAYER_NAME",
              right_on="NAME", how="left").drop(columns=["NAME"])

In [5]:
df["SHOT_ZONE_BASIC"].unique()

array(['Restricted Area', 'Right Corner 3', 'Left Corner 3',
       'Above the Break 3', 'In The Paint (Non-RA)', 'Mid-Range',
       'Backcourt'], dtype=object)

In [6]:
df["SHOT_ZONE_AREA"].unique()

array(['Center(C)', 'Right Side(R)', 'Left Side(L)',
       'Left Side Center(LC)', 'Right Side Center(RC)', 'Back Court(BC)'],
      dtype=object)

In [7]:
equipos = df["TEAM_NAME"].unique()
equipos

array(['Toronto Raptors', 'New York Knicks', 'Miami Heat', 'Utah Jazz',
       'Memphis Grizzlies', 'Minnesota Timberwolves', 'Phoenix Suns',
       'Cleveland Cavaliers', 'New Orleans Pelicans', 'Milwaukee Bucks',
       'Orlando Magic', 'Washington Wizards', 'Portland Trail Blazers',
       'Detroit Pistons', 'Charlotte Hornets', 'Philadelphia 76ers',
       'Boston Celtics', 'San Antonio Spurs', 'Sacramento Kings',
       'Brooklyn Nets', 'LA Clippers', 'Oklahoma City Thunder',
       'Atlanta Hawks', 'Chicago Bulls', 'Denver Nuggets',
       'Houston Rockets', 'Indiana Pacers', 'Dallas Mavericks',
       'Los Angeles Lakers', 'Golden State Warriors'], dtype=object)

## ARREGLAR COLUMNA VISITANTE Y LOCAL: AWAY_TEAM_NAME // HOME_TEAM_NAME

In [8]:
# Diccionario que mapea los nombres completos de los equipos a sus abreviaturas
team_abbr = {
    'Toronto Raptors': 'TOR', 'New York Knicks': 'NYK', 'Miami Heat': 'MIA', 'Utah Jazz': 'UTA',
    'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN', 'Phoenix Suns': 'PHX', 
    'Cleveland Cavaliers': 'CLE', 'New Orleans Pelicans': 'NOP', 'Milwaukee Bucks': 'MIL',
    'Orlando Magic': 'ORL', 'Washington Wizards': 'WAS', 'Portland Trail Blazers': 'POR',
    'Detroit Pistons': 'DET', 'Charlotte Hornets': 'CHA', 'Philadelphia 76ers': 'PHI',
    'Boston Celtics': 'BOS', 'San Antonio Spurs': 'SAS', 'Sacramento Kings': 'SAC',
    'Brooklyn Nets': 'BKN', 'LA Clippers': 'LAC', 'Oklahoma City Thunder': 'OKC', 
    'Atlanta Hawks': 'ATL', 'Chicago Bulls': 'CHI', 'Denver Nuggets': 'DEN',
    'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND', 'Dallas Mavericks': 'DAL',
    'Los Angeles Lakers': 'LAL', 'Golden State Warriors': 'GSW'
}

In [9]:
# Función para asignar las abreviaturas de los equipos según la ubicación
def asignar_abreviatura(row):
    team_name = row['TEAM_NAME']
    
    # Si el nombre del equipo está en el diccionario de abreviaturas
    if team_name in team_abbr:
        player_abbr = team_abbr[team_name]
        
        # Si el jugador está en el equipo visitante
        if row['LOCATION'] == 'AWAY':
            away_abbr = player_abbr  # La abreviatura del equipo del jugador va en AWAY_TEAM_NAME
            # La abreviatura del equipo rival va en HOME_TEAM_NAME (el que no sea el del jugador)
            home_abbr = row['VTM'] if row['VTM'] != away_abbr else row['HTM']
            return pd.Series([away_abbr, home_abbr], index=['AWAY_TEAM_NAME', 'HOME_TEAM_NAME'])
        
        # Si el jugador está en el equipo local
        elif row['LOCATION'] == 'HOME':
            home_abbr = player_abbr  # La abreviatura del equipo del jugador va en HOME_TEAM_NAME
            # La abreviatura del equipo rival va en AWAY_TEAM_NAME (el que no sea el del jugador)
            away_abbr = row['VTM'] if row['VTM'] != home_abbr else row['HTM']
            return pd.Series([home_abbr, away_abbr], index=['HOME_TEAM_NAME', 'AWAY_TEAM_NAME'])
    else:
        # Si no se encuentra la abreviatura del equipo, devolver None
        return pd.Series([None, None], index=['AWAY_TEAM_NAME', 'HOME_TEAM_NAME'])

# Aplicamos la función a todas las filas y sobrescribimos las columnas AWAY_TEAM_NAME y HOME_TEAM_NAME
df[['AWAY_TEAM_NAME', 'HOME_TEAM_NAME']] = df.apply(asignar_abreviatura, axis=1)

# Verificar que el dataframe ha sido actualizado
print(f'Número de filas después de la actualización: {df.shape[0]}')



# Para visualizar algunas filas y verificar
print(df.head())

Número de filas después de la actualización: 259017
           GRID_TYPE   GAME_ID  GAME_EVENT_ID  PLAYER_ID       PLAYER_NAME  \
0  Shot Chart Detail  22300031            130    1630173  Precious Achiuwa   
1  Shot Chart Detail  22300031            130    1630173  Precious Achiuwa   
2  Shot Chart Detail  22300031            148    1630173  Precious Achiuwa   
3  Shot Chart Detail  22300031            148    1630173  Precious Achiuwa   
4  Shot Chart Detail  22300031            195    1630173  Precious Achiuwa   

      TEAM_ID        TEAM_NAME  PERIOD  MINUTES_REMAINING  SECONDS_REMAINING  \
0  1610612761  Toronto Raptors       1                  1                 49   
1  1610612761  Toronto Raptors       1                  1                 49   
2  1610612761  Toronto Raptors       1                  0                 11   
3  1610612761  Toronto Raptors       1                  0                 11   
4  1610612761  Toronto Raptors       2                  8                 49   

## TRADUCCION NOMBRE POSICIONES

In [10]:
#CAMBIAR NOMBRE POSICIONES
df["POS"] = df["POS"].replace({
    "C": "P",
    "G": "B",
    "F-C": "AP",
    "C-F": "AP",
    "F-G": "ES",
    "G-F": "ES",
    "F": "A"
})


## VER Y CAMBIAR LOS NULL

In [11]:
df.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM', 'player_name', 'Season Type', 'HOME_TEAM_NAME',
       'AWAY_TEAM_NAME', 'LOCATION', 'POS'],
      dtype='object')

In [12]:
jugadores_sin_pos = df[df["POS"].isna()]["PLAYER_NAME"].unique()
jugadores_sin_pos

array(['Patrick Baldwin Jr.', 'D’Moi Hodge', 'GG Jackson II', 'AJ Lawson',
       'Craig Porter Jr.'], dtype=object)

In [13]:
#CAMBIAR NULLS A MANO
df.loc[df["PLAYER_NAME"] == "Patrick Baldwin Jr.", "POS"] = "A"
df.loc[df["PLAYER_NAME"] == "D’Moi Hodge", "POS"] = "ES"
df.loc[df["PLAYER_NAME"] == "GG Jackson II", "POS"] = "A"
df.loc[df["PLAYER_NAME"] == "AJ Lawson", "POS"] = "ES"
df.loc[df["PLAYER_NAME"] == "Craig Porter Jr.", "POS"] = "B"

## CONVERSIÓN A METROS

In [14]:
#  COLUMNA SHOT_DISTANCE
df["SHOT_DISTANCE_METERS"] = (df["SHOT_DISTANCE"] * 0.3048).round(2)  #me quedo con dos decimales

In [15]:
#  columna SHOT_ZONE_RANGE (más complicada porque mezcla los pies con texto)

# Función para convertir la distancia en pies a metros
def convertir_a_metros(valor):
    # Comprobar si la cadena contiene "ft"
    if isinstance(valor, str) and "ft" in valor:
        if "Less Than" in valor:
            # Para "Less Than 8 ft."  
            return "Less Than 2.44 m"
        elif "24+" in valor:
            # Para "24+ ft."  
            return "More Than 7.32 m"
        elif "16-24" in valor:
            # Para "16-24 ft."  
            return "4.88-7.32 m"
        elif "8-16" in valor:
            # Para "8-16 ft."  
            return "2.44-4.88 m"
    return valor


In [16]:
# Aplicar la conversión en la columna 'SHOT_ZONE_RANGE'
df['SHOT_ZONE_RANGE_METERS'] = df['SHOT_ZONE_RANGE'].apply(convertir_a_metros)
# Verificar los resultados
print(df[['SHOT_ZONE_RANGE', 'SHOT_ZONE_RANGE_METERS']].head(56))

    SHOT_ZONE_RANGE SHOT_ZONE_RANGE_METERS
0   Less Than 8 ft.       Less Than 2.44 m
1   Less Than 8 ft.       Less Than 2.44 m
2           24+ ft.       More Than 7.32 m
3           24+ ft.       More Than 7.32 m
4           24+ ft.       More Than 7.32 m
5           24+ ft.       More Than 7.32 m
6   Less Than 8 ft.       Less Than 2.44 m
7   Less Than 8 ft.       Less Than 2.44 m
8           24+ ft.       More Than 7.32 m
9           24+ ft.       More Than 7.32 m
10  Less Than 8 ft.       Less Than 2.44 m
11  Less Than 8 ft.       Less Than 2.44 m
12  Less Than 8 ft.       Less Than 2.44 m
13  Less Than 8 ft.       Less Than 2.44 m
14  Less Than 8 ft.       Less Than 2.44 m
15  Less Than 8 ft.       Less Than 2.44 m
16  Less Than 8 ft.       Less Than 2.44 m
17  Less Than 8 ft.       Less Than 2.44 m
18  Less Than 8 ft.       Less Than 2.44 m
19  Less Than 8 ft.       Less Than 2.44 m
20  Less Than 8 ft.       Less Than 2.44 m
21  Less Than 8 ft.       Less Than 2.44 m
22  Less Th

## CAMBIAR FORMATO FECHA

In [17]:
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'], format='%Y%m%d')

## ELIMINAR COLUMNAS

In [18]:
df.drop(columns=["HTM", "VTM", "SHOT_DISTANCE", "SHOT_ZONE_RANGE",
                 "player_name", "GRID_TYPE", "GAME_EVENT_ID"], inplace=True)

## ELMINAR TIROS DUPLICADOS

In [19]:
print(f"Número de filas antes de eliminar duplicados: {len(df)}")

Número de filas antes de eliminar duplicados: 259017


In [20]:
# aqui comparo todas las filas con valores (quitando el GAME_EVENT_ID que sean completamente identicos y elimino esas filas)
df_cleaned = df.drop_duplicates(subset=list(df.columns.difference(['GAME_EVENT_ID']))).copy() #todo igual menos GAME_EVENT_ID
print(f"Número de filas después de eliminar duplicados: {len(df_cleaned)}")

Número de filas después de eliminar duplicados: 233999


In [21]:
# filas eliminadas:
259017 - 233999

25018

In [22]:
duplicates = df_cleaned.duplicated(keep=False)  # keep=False muestra todos los duplicados
print(df_cleaned[duplicates])  # Si hay filas repetidas, aquí las verás

Empty DataFrame
Columns: [GAME_ID, PLAYER_ID, PLAYER_NAME, TEAM_ID, TEAM_NAME, PERIOD, MINUTES_REMAINING, SECONDS_REMAINING, EVENT_TYPE, ACTION_TYPE, SHOT_TYPE, SHOT_ZONE_BASIC, SHOT_ZONE_AREA, LOC_X, LOC_Y, SHOT_ATTEMPTED_FLAG, SHOT_MADE_FLAG, GAME_DATE, Season Type, HOME_TEAM_NAME, AWAY_TEAM_NAME, LOCATION, POS, SHOT_DISTANCE_METERS, SHOT_ZONE_RANGE_METERS]
Index: []

[0 rows x 25 columns]


In [23]:
# Crear una nueva columna NEW_GAME_EVENT_ID basada en todas las columnas  menos game event id

df_cleaned.loc[:, 'NEW_GAME_EVENT_ID'] = df_cleaned.groupby(
    ['GAME_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING',
       'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC',
       'SHOT_ZONE_AREA', 'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG',
       'SHOT_MADE_FLAG', 'GAME_DATE', 'Season Type', 'HOME_TEAM_NAME',
       'AWAY_TEAM_NAME', 'LOCATION', 'POS', 'SHOT_DISTANCE_METERS',
       'SHOT_ZONE_RANGE_METERS']
).ngroup() + 1 

In [24]:
# vuelvo a verificar si hay duplicados

In [25]:
event_counts = df_cleaned['NEW_GAME_EVENT_ID'].value_counts()
duplicate_ids = event_counts[event_counts > 1].index

print(df_cleaned[df_cleaned['NEW_GAME_EVENT_ID'].isin(duplicate_ids)])

Empty DataFrame
Columns: [GAME_ID, PLAYER_ID, PLAYER_NAME, TEAM_ID, TEAM_NAME, PERIOD, MINUTES_REMAINING, SECONDS_REMAINING, EVENT_TYPE, ACTION_TYPE, SHOT_TYPE, SHOT_ZONE_BASIC, SHOT_ZONE_AREA, LOC_X, LOC_Y, SHOT_ATTEMPTED_FLAG, SHOT_MADE_FLAG, GAME_DATE, Season Type, HOME_TEAM_NAME, AWAY_TEAM_NAME, LOCATION, POS, SHOT_DISTANCE_METERS, SHOT_ZONE_RANGE_METERS, NEW_GAME_EVENT_ID]
Index: []

[0 rows x 26 columns]


In [26]:
#meuvo al columna NEW_EVENT_GAME_ID a la segunda posición, como estaba la otra:
# Guardamos las columnas en una lista
cols = df_cleaned.columns.tolist()

# Quitamos la columna 'NEW_GAME_EVENT_ID' de su posición original
cols.remove('NEW_GAME_EVENT_ID')

# Insertamos la columna 'NEW_GAME_EVENT_ID' en la segunda posición
cols.insert(1, 'NEW_GAME_EVENT_ID')

# Reordenamos el dataframe según esta nueva lista de columnas
df_cleaned = df_cleaned[cols]

In [27]:
# Paso Season Type a mayuscula
df.columns = df.columns.str.upper()

In [28]:
# guardo el csv
df_cleaned.to_csv('../data/NBA_BBDD_CLEAN.csv', index=False)

In [3]:
df = pd.read_csv("../data/NBA_BBDD_CLEAN.csv")

In [4]:
df.columns

Index(['GAME_ID', 'NEW_GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING',
       'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC',
       'SHOT_ZONE_AREA', 'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG',
       'SHOT_MADE_FLAG', 'GAME_DATE', 'Season Type', 'HOME_TEAM_NAME',
       'AWAY_TEAM_NAME', 'LOCATION', 'POS', 'SHOT_DISTANCE_METERS',
       'SHOT_ZONE_RANGE_METERS'],
      dtype='object')

In [4]:
df['NEW_GAME_EVENT_ID'].nunique()

233999

In [17]:
lanzamientos = df.groupby("PLAYER_NAME")["SHOT_ATTEMPTED_FLAG"].sum()

In [18]:
lanzamientos.sort_values(ascending= False).head(5)

PLAYER_NAME
Luka Doncic                2156
Jalen Brunson              1988
Anthony Edwards            1882
Jayson Tatum               1798
Shai Gilgeous-Alexander    1711
Name: SHOT_ATTEMPTED_FLAG, dtype: int64

---------------------------------------
------------------------------------------


# RESUMEN TRABAJO LIMPIEZA

### PROBLEMAS DEL DATASET ORIGINAL


***Datos repetidos***: Probablemente provenientes de la unión de varias tablas.

***Inconsistencias en columnas:***

- Columnas contradictorias (ejemplo: partido en casa o fuera).

IDs de tiros repetidos, lo que generaba duplicados.

***Falta de estandarización:***

- Formatos de texto inconsistentes (mayúsculas/minúsculas).

- Columnas con información redundante o poco útil.

### TRANSFORMACIÓN


***Unión con otra BBDD:***

- Se agregaron columnas para identificar si el equipo era local o visitante.

- Esto ayudó a enriquecer la información y mejorar la consistencia.



### TRABAJO DE LIMPIEZA

***Eliminación de columnas innecesarias:***

- Se quitaron columnas que no aportaban información útil.

***Estandarización de formatos:***

- Unificación de mayúsculas/minúsculas en los títulos de las columnas.

- Cambio de formato a fechas en columnas relevantes.

***Traducción y normalización: ***

- Traducción de posiciones de jugadores al español para facilitar su comprensión.

***Tratamiento de valores nulos:***

- Relleno de posiciones de jugadores que tenían valores nulos.

***Conversión de unidades:***

- Cambio de pies a metros en columnas de distancias.

- Corrección de columnas que mezclaban texto con distancias (ejemplo: "2 m y 44"  "2.44 metros")

***Eliminación de duplicados:***

- ***Problema***: Los IDs de tiros estaban mal asignados, lo que generaba duplicados.

- ***Solución:***

        - Comparación de todos los valores de los tiros (excepto el ID) para identificar duplicados.

        - Eliminación de aproximadamente 25,000 filas duplicadas.

        - Asignación de un nuevo ID único a cada tiro.

        - Resultado: El dataset final quedó con aproximadamente 234,000 filas.




### RESULTADO FINAL


- Dataset limpio, consistente y listo para su análisis.

- Mayor claridad en la información, con formatos estandarizados y sin duplicados.

- Información enriquecida con la unión de la segunda BBDD.